In [1]:
import pandas as pd
from datetime import datetime

df = pd.read_csv('log-info.csv')

df.reset_index(inplace=True)
df

,index,userid,sessionid,timestamp,action,value,category,age,gender,city,point
0,0,user_10000,1,2021-12-28T02:33:15,mainpage,0.000000,None,48,М,Омск,"[54.989347,73.368221]"
1,1,user_10000,1,2021-12-28T02:33:29,search,0.000000,Продовольственные товары,48,М,Омск,"[54.989347,73.368221]"
2,2,user_10000,1,2021-12-28T02:33:47,product,0.000000,Продовольственные товары,48,М,Омск,"[54.989347,73.368221]"
3,3,user_10000,1,2021-12-28T02:33:59,mainpage,0.000000,None,48,М,Омск,"[54.989347,73.368221]"
4,4,user_10000,1,2021-12-28T02:34:10,category,0.000000,Товары для детей,48,М,Омск,"[54.989347,73.368221]"
...,...,...,...,...,...,...,...,...,...,...,...
14553412,14553412,user_14999,529,2025-01-02T04:00:01,cart,23613.725533,Продовольственные товары,34,Ж,Челябинск,"[55.159902,61.402554]"
14553413,14553413,user_14999,529,2025-01-02T04:00:45,checkout,992.000000,None,34,Ж,Челябинск,"[55.159902,61.402554]"
14553414,14553414,user_14999,529,2025-01-02T04:01:08,confirmation,992.000000,None,34,Ж,Челябинск,"[55.159902,61.402554]"
14553415,14553415,user_14999,530,2025-01-03T04:01:15,search,0.000000,Продовольственные товары,34,Ж,Челябинск,"[55.159902,61.402554]"


In [2]:
first = df.groupby('userid')['index'].min().reset_index()
first['join'] = 1
first.drop(columns='userid', inplace=True)
first

,index,join
0,0,1
1,3034,1
2,5979,1
3,8975,1
4,11746,1
...,...,...
4995,14538777,1
4996,14541741,1
4997,14544886,1
4998,14547654,1


In [3]:
last = df.groupby('userid')['index'].max().reset_index()
last['leave'] = 1
last.drop(columns='userid', inplace=True)
last

,index,leave
0,3033,1
1,5978,1
2,8974,1
3,11745,1
4,14720,1
...,...,...
4995,14541740,1
4996,14544885,1
4997,14547653,1
4998,14550822,1


In [4]:
df = df.merge(first, how='left', on='index').fillna(0) \
    .merge(last, how='left', on='index').fillna(0) \
    .drop(columns='index') \
    .astype({'join' : 'int', 'leave' : 'int'})

In [5]:
df.sort_values(by='timestamp', inplace=True)
df['rjoin'] = df['join'].cumsum()
df['rleave'] = df['leave'].cumsum()
df

,userid,sessionid,timestamp,action,value,category,age,gender,city,point,join,leave,rjoin,rleave
4555707,user_11565,1,2021-12-26T21:01:02,product,0.000000,Электроника и бытовая техника,40,Ж,Челябинск,"[55.159902,61.402554]",1,0,1,0
12332526,user_14237,1,2021-12-26T21:03:37,category,0.000000,Красота и здоровье,28,Ж,Казань,"[55.796127,49.106414]",1,0,2,0
12332527,user_14237,1,2021-12-26T21:03:47,search,0.000000,Красота и здоровье,28,Ж,Казань,"[55.796127,49.106414]",0,0,2,0
12332528,user_14237,1,2021-12-26T21:04:02,product,0.000000,Красота и здоровье,28,Ж,Казань,"[55.796127,49.106414]",0,0,2,0
1207724,user_10415,1,2021-12-26T21:07:35,search,0.000000,Одежда и обувь,32,М,Самара,"[53.195878,50.100202]",1,0,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4587662,user_11575,567,2025-01-03T21:03:12,category,0.000000,Товары для детей,39,М,Казань,"[55.796127,49.106414]",0,0,5000,4999
4587663,user_11575,567,2025-01-03T21:03:33,product,0.000000,Товары для детей,39,М,Казань,"[55.796127,49.106414]",0,0,5000,4999
4587664,user_11575,567,2025-01-03T21:03:54,category,0.000000,Электроника и бытовая техника,39,М,Казань,"[55.796127,49.106414]",0,0,5000,4999
4587665,user_11575,567,2025-01-03T21:04:21,cart,967.042253,Продовольственные товары,39,М,Казань,"[55.796127,49.106414]",0,0,5000,4999


In [6]:
first_action = df.groupby('userid')['timestamp'].min().reset_index().rename(columns={'timestamp' : 'first_action'})
df = df.merge(first_action, how='left', on='userid')
df

,userid,sessionid,timestamp,action,value,category,age,gender,city,point,join,leave,rjoin,rleave,first_action
0,user_11565,1,2021-12-26T21:01:02,product,0.000000,Электроника и бытовая техника,40,Ж,Челябинск,"[55.159902,61.402554]",1,0,1,0,2021-12-26T21:01:02
1,user_14237,1,2021-12-26T21:03:37,category,0.000000,Красота и здоровье,28,Ж,Казань,"[55.796127,49.106414]",1,0,2,0,2021-12-26T21:03:37
2,user_14237,1,2021-12-26T21:03:47,search,0.000000,Красота и здоровье,28,Ж,Казань,"[55.796127,49.106414]",0,0,2,0,2021-12-26T21:03:37
3,user_14237,1,2021-12-26T21:04:02,product,0.000000,Красота и здоровье,28,Ж,Казань,"[55.796127,49.106414]",0,0,2,0,2021-12-26T21:03:37
4,user_10415,1,2021-12-26T21:07:35,search,0.000000,Одежда и обувь,32,М,Самара,"[53.195878,50.100202]",1,0,3,0,2021-12-26T21:07:35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14553412,user_11575,567,2025-01-03T21:03:12,category,0.000000,Товары для детей,39,М,Казань,"[55.796127,49.106414]",0,0,5000,4999,2021-12-28T07:28:19
14553413,user_11575,567,2025-01-03T21:03:33,product,0.000000,Товары для детей,39,М,Казань,"[55.796127,49.106414]",0,0,5000,4999,2021-12-28T07:28:19
14553414,user_11575,567,2025-01-03T21:03:54,category,0.000000,Электроника и бытовая техника,39,М,Казань,"[55.796127,49.106414]",0,0,5000,4999,2021-12-28T07:28:19
14553415,user_11575,567,2025-01-03T21:04:21,cart,967.042253,Продовольственные товары,39,М,Казань,"[55.796127,49.106414]",0,0,5000,4999,2021-12-28T07:28:19


In [7]:
df.to_csv('log-info-ext.csv', index=False)